In [1]:
%load_ext autoreload
%autoreload 2
from geolifeclef.utils import get_spark
from pyspark.sql import functions as F

spark = get_spark()
data_root = "/mnt/data/geolifeclef-2024/data"

edges = spark.read.parquet(f"{data_root}/processed/geolsh_graph/v1/edges")
edges.printSchema()

24/04/06 11:11:08 WARN Utils: Your hostname, daphne-major resolves to a loopback address: 127.0.1.1; using 172.28.199.217 instead (on interface eth0)
24/04/06 11:11:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/06 11:11:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/06 11:11:10 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


root
 |-- srcDataset: string (nullable = true)
 |-- srcSurveyId: integer (nullable = true)
 |-- srcSpeciesId: double (nullable = true)
 |-- dstDataset: string (nullable = true)
 |-- dstSurveyId: integer (nullable = true)
 |-- dstSpeciesId: double (nullable = true)
 |-- euclidean: double (nullable = true)
 |-- threshold: integer (nullable = true)



In [2]:
edges.count()

1116195844

In [3]:
edges.select("euclidean").describe().show()

+-------+------------------+
|summary|         euclidean|
+-------+------------------+
|  count|        1116195844|
|   mean|31299.572400645804|
| stddev|29892.019010722808|
|    min|               0.0|
|    max| 99999.99986340413|
+-------+------------------+



In [4]:
# let's discretize thresholds into buckets and see how big they are
from pyspark.sql import functions as F

edges.select(
    F.round("euclidean", -4).alias("euclidean_10km"),
).groupBy(
    "euclidean_10km"
).count().orderBy("euclidean_10km").show()

+--------------+---------+
|euclidean_10km|    count|
+--------------+---------+
|           0.0|318011094|
|       10000.0|146144208|
|       20000.0|117410260|
|       30000.0| 99743946|
|       40000.0| 85627662|
|       50000.0| 75692624|
|       60000.0| 73006062|
|       70000.0| 65306260|
|       80000.0| 58686854|
|       90000.0| 52680442|
|      100000.0| 23886432|
+--------------+---------+



In [5]:
# let's compute some basic graph statistics for survey-species and species-species
def degree(df, src, dst):
    return df.groupBy(src, dst).agg(F.count("*").alias("n")).groupBy(src).count()


degree(edges, "srcSurveyId", "dstSpeciesId").describe().show()
degree(edges, "srcSpeciesId", "dstSpeciesId").describe().show()

+-------+------------------+------------------+
|summary|       srcSurveyId|             count|
+-------+------------------+------------------+
|  count|           3919238|           3919238|
|   mean|1959826.7707176753| 60.60069533924707|
| stddev|1131506.4547204622|57.937901884098785|
|    min|                 1|                 1|
|    max|           3919662|               370|
+-------+------------------+------------------+



+-------+-----------------+------------------+
|summary|     srcSpeciesId|             count|
+-------+-----------------+------------------+
|  count|            10358|             10359|
|   mean|5626.499903456266|1065.6767062457766|
| stddev|3250.528165571007| 1307.583074547074|
|    min|              0.0|                 1|
|    max|          11254.0|              6134|
+-------+-----------------+------------------+



In [6]:
# let's do it again for a different threshold

degree(
    edges.where("euclidean < 10000"), "srcSurveyId", "dstSpeciesId"
).describe().show()
degree(
    edges.where("euclidean < 10000"), "srcSpeciesId", "dstSpeciesId"
).describe().show()

+-------+------------------+------------------+
|summary|       srcSurveyId|             count|
+-------+------------------+------------------+
|  count|           3919238|           3919238|
|   mean|1959826.7707176753| 19.84241885795147|
| stddev|1131506.4547204617|25.982557877884812|
|    min|                 1|                 1|
|    max|           3919662|               273|
+-------+------------------+------------------+



+-------+-----------------+-----------------+
|summary|     srcSpeciesId|            count|
+-------+-----------------+-----------------+
|  count|            10358|            10359|
|   mean|5626.499903456266|698.3180808958393|
| stddev| 3250.52816557101|955.1556924836394|
|    min|              0.0|                1|
|    max|          11254.0|             5083|
+-------+-----------------+-----------------+

